In [40]:
import pandas as pd

# create 2011 dataframe

df = pd.DataFrame([])

states = {
    2: 'NSW',
    3: 'Vic',
    4: 'Qld',
    5: 'SA',
    6: 'WA',
    7: 'Tas',
    8: 'NT',
    9: 'ACT'
}

for n in range(2,10):
    statedf = pd.read_excel('32180ds0005_2005-15.xls',sheetname='Table '+str(n),header=6)
    statedf = statedf[2011].reset_index().dropna(subset=['level_0',2011])[1:]
    statedf['state'] = states[n]
    df = df.append(statedf)
df.head()

,level_0,level_1,2011,state
2,10001,Albury,74360,NSW
3,10002,Auburn,96953,NSW
4,10003,Ballina,68893,NSW
5,10004,Balmain,80387,NSW
6,10005,Bankstown,84832,NSW


In [50]:
cor = pd.read_excel('CG_SED_2011_SED_2016.xls',sheetname='Table 3',header=5).dropna()
cor = cor[~cor['SED_NAME_2011'].str.contains('No usual address')]
cor = cor[~cor['SED_NAME_2011'].str.contains('Migratory')]
cor = cor[~cor['SED_NAME_2011'].str.contains('Unclassified')]


cor.head()

,SED_CODE_2011,SED_NAME_2011,SED_CODE_2016,SED_NAME_2016,RATIO,PERCENTAGE
1,10001,Albury,10001.0,Albury,1.000000,100.000000
2,10002,Auburn,10002.0,Auburn,0.801217,80.121679
3,10002,Auburn,10005.0,Bankstown,0.106911,10.691097
4,10002,Auburn,10043.0,Lakemba,0.091872,9.187224
5,10003,Ballina,10003.0,Ballina,1.000000,100.000000


In [81]:
merged = pd.merge(cor,df,left_on='SED_CODE_2011',right_on='level_0')
merged['population'] = (merged['RATIO'] * merged[2011]).astype(int)
pop = merged[['SED_CODE_2016','SED_NAME_2016','population']].groupby(['SED_CODE_2016','SED_NAME_2016']).sum().reset_index()
pop.head()

,SED_CODE_2016,SED_NAME_2016,population
0,10001.0,Albury,77095
1,10002.0,Auburn,91658
2,10003.0,Ballina,71395
3,10004.0,Balmain,79579
4,10005.0,Bankstown,82306


In [83]:
# this gives us a base - that is, a base population conversion from the correspondence file.


elect = cor[['SED_CODE_2016','SED_NAME_2016']]
base = pd.merge(pop,elect,right_on='SED_CODE_2016',left_on='SED_CODE_2016').drop_duplicates()
base.head()

,SED_CODE_2016,SED_NAME_2016_x,population,SED_NAME_2016_y
0,10001.0,Albury,77095,Albury
2,10002.0,Auburn,91658,Auburn
4,10003.0,Ballina,71395,Ballina
6,10004.0,Balmain,79579,Balmain
8,10005.0,Bankstown,82306,Bankstown


In [99]:
# now we need to go through the state legislatures one by one

# Here is a summary of the different legislatures

summary = pd.DataFrame([
    ['NSW','Upper','Legislative Council',42,'Single state'],
    ['NSW','Lower','Legislative Assembly',93,'Proportional'],
    ['Vic','Upper','Legislative Council',40,'8 regions'],
    ['Vic','Lower','Legislative Assembly',88,'Proportional'],
    ['Qld','Lower','Legislative Assembly',89,'Proportional'],
    ['SA','Upper','Legislative Council',22,'Single state'],
    ['SA','Lower','House of Assembly',47,'Proportional'],
    ['WA','Upper','Legislative Council',36,'6 regions'],
    ['WA','Lower','Legislative Assembly',59,'Proportional'],
    ['Tas','Upper','Legislative Council',15,'Proportional'],
    ['Tas','Lower','House of Assembly',25,'5 regions'],
    ['NT','Lower','Legislative Assembly',25,'Proportional'],
    ['ACT','Lower','Legislative Assembly',25,'5 regions'],
])
summary.columns = ['State','Type','Name','Reps','Electorate']
summary

,State,Type,Name,Reps,Electorate
0,NSW,Upper,Legislative Council,42,Single state
1,NSW,Lower,Legislative Assembly,93,Proportional
2,Vic,Upper,Legislative Council,40,8 regions
3,Vic,Lower,Legislative Assembly,88,Proportional
4,Qld,Lower,Legislative Assembly,89,Proportional
5,SA,Upper,Legislative Council,22,Single state
6,SA,Lower,House of Assembly,47,Proportional
7,WA,Upper,Legislative Council,36,6 regions
8,WA,Lower,Legislative Assembly,59,Proportional
9,Tas,Upper,Legislative Council,15,Proportional


In [104]:
final[final['SED_NAME_2016_x'].str.contains(" ")]

,SED_CODE_2016,SED_NAME_2016_x,population,SED_NAME_2016_y
18,10008.0,Baulkham Hills,75072,Baulkham Hills
24,10011.0,Blue Mountains,72868,Blue Mountains
39,10016.0,Castle Hill,74985,Castle Hill
50,10020.0,Coffs Harbour,70967,Coffs Harbour
66,10027.0,East Hills,74902,East Hills
105,10042.0,Lake Macquarie,71288,Lake Macquarie
112,10044.0,Lane Cove,78036,Lane Cove
124,10048.0,Macquarie Fields,78652,Macquarie Fields
135,10054.0,Mount Druitt,86100,Mount Druitt
142,10057.0,Myall Lakes,70481,Myall Lakes


In [89]:
tas = final[(final['SED_CODE_2016']>60000)&(final['SED_CODE_2016']<70000)]
tas['Upper'] = tas['SED_NAME_2016_x'].apply(lambda x: x.split("(")[1][:-1])
tas

/Users/jacksongs/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,SED_CODE_2016,SED_NAME_2016_x,population,SED_NAME_2016_y,Upper
790,60101.0,Bass (Apsley),10734,Bass (Apsley),Apsley
791,60106.0,Bass (Launceston),33535,Bass (Launceston),Launceston
792,60112.0,Bass (Rosevears),23826,Bass (Rosevears),Rosevears
793,60115.0,Bass (Windermere),35654,Bass (Windermere),Windermere
795,60207.0,Braddon (Mersey),33710,Braddon (Mersey),Mersey
797,60208.0,Braddon (Montgomery),33438,Braddon (Montgomery),Montgomery
798,60209.0,Braddon (Murchison),35019,Braddon (Murchison),Murchison
799,60214.0,Braddon (Western Tiers),228,Braddon (Western Tiers),Western Tiers
800,60302.0,Denison (Derwent),9438,Denison (Derwent),Derwent
801,60303.0,Denison (Elwick),35645,Denison (Elwick),Elwick


In [93]:
tasupper = tas.groupby('Upper').sum()
tasupper

,SED_CODE_2016,population
Upper,,
Apsley,120602.0,31634
Derwent,181206.0,36978
Elwick,60303.0,35645
Hobart,60304.0,33775
Huon,120710.0,34637
Launceston,60106.0,33535
Mersey,60207.0,33710
Montgomery,60208.0,33438
Murchison,60209.0,35019
